In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [ ]:
# Load your data (replace 'your_data.csv' with your actual file)
data = pd.read_csv(r"SafeBite\food-contamination-data-cleaned-2.csv")

# Define X and Y
X = data.drop('ResultValue', axis=1) # All columns except ResultValue
y = data['ResultValue'] # The ResultValue column

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(exclude=['object']).columns.tolist()

In [6]:
# Create transformers for categorical and numerical columns
numerical_transformer = 'passthrough' # No transformation needed for numerical

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Create a pipeline with preprocessing and the linear regression model
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', LinearRegression())])

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

In [8]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 1704.746408723103
R-squared: 0.04655158721223818


Since the r2 value is very low, the data is not related in a linear manner. it must be regularised. L1 / Lasso Regularisation is being used below

In [10]:
# Create Lasso model in a pipeline
lasso_model = Pipeline(steps=[('preprocessor', preprocessor), 
                              ('regressor', Lasso())])

# Define the parameter grid for alpha tuning
param_grid = {'regressor__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Create GridSearchCV object for cross-validation
grid_search_lasso = GridSearchCV(lasso_model, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the GridSearchCV object
grid_search_lasso.fit(X_train, y_train)

# Print the best alpha value
print(f'Best Lasso alpha: {grid_search_lasso.best_params_["regressor__alpha"]}')

# Get the best Lasso model
best_lasso_model = grid_search_lasso.best_estimator_

c:\Users\Ananya\Documents\GitHub\SafeBite\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23035910.87468704, tolerance: 6983.133609285355
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\Ananya\Documents\GitHub\SafeBite\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22958295.30327101, tolerance: 6951.078707136703
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\Ananya\Documents\GitHub\SafeBite\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22855603.947489653, tolerance: 6909.2075906941345
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\Ananya\D

Best Lasso alpha: 0.001


c:\Users\Ananya\Documents\GitHub\SafeBite\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 28728446.119015217, tolerance: 8680.279789380971
  model = cd_fast.sparse_enet_coordinate_descent(


In [11]:
# Evaluate the best Lasso model on the test set
y_pred_lasso = best_lasso_model.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f'Lasso MSE: {mse_lasso}')
print(f'Lasso R-squared: {r2_lasso}')

Lasso MSE: 1188.1131293703013
Lasso R-squared: 0.3354996546032245
